## Module 1: Prepare Data Using SageMaker Data Wrangler

In this one-hour module,  you will learn how to use SageMaker Data Wrangler to explore, clean, transform, and visualize your data, including techniques such as data aggregation, normalization, and feature engineering. Optionally, you will also learn how to train an auto-ml model directly in Data wrangler and deploy the model to online endpoints or run batch predictions offline.

By the end of this module, you will be able to:

* Understand the basic concepts of data preparation for machine learning
* Use SageMaker Data Wrangler to explore, clean, transform, and visualize data
* Create automate the data preparation workflow jobs to process your data at scale.
* [**Optionally**] create data ML models with no code.

## Navigate to Data Wrangler

Make sure you have completed all the steps in the *0_setup.ipynb notebook*. Now you can navigate to *SageMaker Data Wrangler: Home → Data → Data Wrangler → Import Data*.\

 a. This also creates a .flow file in your current directory.\
 b. [**optional**] Rename to flow file to *5gcell_workshop.flow*.

![Data Wrangler Flow](statics/module_01_dw02.png)
![Data Wrangler Flow](statics/module_01_dw03.png)

## Import Dataset from S3 bucket

1. We start by importing the dataset we previously uploaded to our S3 bucket. Inside Data Wrangler, select **Amazon S3** from Data sources.  The Import a dataset from S3 page will be displayed. 
  
![Import from S3](statics/dw-data-source.png)
    
2. Navigate to the bucket and folder that contains the *5gcell.csv* file.
3. Select the *5gcell.csv* file. You'll see a preview of the data.
4. **Sampling Options**: You have the option to import your entire dataset into Data Wrangler or to sample a portion of it.

> **Note**\
The larger the dataset, the more accurate your analyses and visualizations will be and the longer they may take to render. By importing only a sample, rendering time may improve, but at the possible expense of losing influential data points. Random and stratified sampling strategies may help mitigate issues like these, but this depends on the distribution of the data and your unique use case.
>
> The following sampling settings only apply during interactive mode within Data Wrangler. When exporting (for example, to training or S3), these settings are ignored. If you wish to return a smaller subset of the data when exporting, use the Split Data transform.   
> When importing from Amazon S3, the following sampling options are available:\
> **None** – Import the entire dataset.\
> **First K** – Sample the first K rows of the dataset, where K is an integer that you specify.\
> **Randomized** – Takes a random sample of a size that you specify.\
> **Stratified** – Takes a stratified random sample. A stratified sample preserves the ratio of values in a column.

5. Let's accept all the defaults (First K and 50,000) and click the Import button. Check [Data Wrangler benchmark tests](https://aws.amazon.com/blogs/machine-learning/process-larger-and-wider-datasets-with-amazon-sagemaker-data-wrangler/#:~:text=Data%20Wrangler%20benchmark%20tests&text=This%20dataset's%20expanded%20size%20is,%2Dend%20customer%2Dfacing%20latency.) for its limit. 

### Navigating Data Wrangler Workspace

After importing the data, you will see a summary page 3 tabs: Data, Analysis, Training.

- Data tab summarizes the steps added to the data source at this point of data transformation. Expand the individual steps to modify.
- Analysis tab shows the visualization/report generated.
- You use the training tab to train an AutoML model (we will cover in more details later)
- Use the **< Data flow button** on the top left to get to the main data flow workspace.
  
![DW Workspace](statics/dw-workspace.png)


## Navigating Data Wrangler Workspace

After importing the data, you will see a summary page with 3 tabs: Data, Analysis, Training

* Data tab summarizes the steps add to the data sources at this point of data transformation. Expand the individual steps to modify.
* Analysis tab shows are the visualization/report generated
* You use the training tab to training an AutoML model (we will cover in more details later)

![DW03](statics/module_01_dw03.png)


## Custom Transform"
---

To determine good 5G accessibility, we will use *5g_sgnb_abnormal_release_rate_num*, which represents the likelihood of connectivity drops. Any record with *5g_sgnb_abnormal_release_rate_num* > 0 is considered anomaly. Then we will train a classification model that can predict the anomaly based on input features. This use case is part of the 5G performance observability initiative, aimed at predicting any potential loss of connectivity to the 5G radio network in the next hour, helping to ensure a seamless and uninterrupted user experience.

We first use **Custom transform** in Data Wrangler to label the anomaly. For 5g_sgnb_abnormal_release_rate_num > 0, we label to 1. Otherwise, label to 0.

Data Wrangler includes built-in transforms, which you can use to transform columns without any code. You can also add custom transformations using PySpark, Python (User-Defined Function), Pandas, and PySpark SQL. Some transforms operate in place, while others create a new output column in your dataset.
You can import the popular libraries with an import statement in the custom transform code block, such as the following:

* Numpy version 1.19.0
* Scikit-learn version 0.23.2
* Scipy version 1.5.4
* Pandas version 1.0.3
* Pyspark version 3.0.0

To add a custom transform, click on the plus sign next to **Data Type** step.

![DW Workspace](statics/add-custom-xform.png)

Steps for custom transformation are:

1. Click **Add step**.
2. Select **Custom transform**.
3. **Name** put Impute anomaly.
4. Select **Python (Pandas)**.
5. Place following code snippet into the code box. 

```
# Table is available as variable `df`
df['anomaly'] = 0
df.loc[df['5g_sgnb_abnormal_release_rate_num']>0, 'anomaly']= 1
df = df.drop(['5g_sgnb_abnormal_release_rate_num'], axis=1)
```
  
![DW Workspace](statics/custom-xform.png)

## Get Insights On Data and Data Quality

1. Use the **< Data flow** button on the top left to get to the main data flow workspace.
2. To get some insights on the data we've just imported, click the “+” icon next to the **Data types** node in the Data Flow diagram, select **Get data insights**. 
  
![Data Insights](statics/data-insights.png)

This is a shortcut that takes us to the analysis page where we are provided with a list of various analysis types to choose and apply.
    
![Data Insights](statics/data-insights-2.png)
    
3. By default, the selected **Analysis Type** is **Data Quality and Insights Report**.
> Data Quality and Insights Report is a quick way to get a better understanding of your dataset. It generates a comprehensive report of your data across the following topics: Summary, Duplicate Rows, Anomalous Samples, Target Column, Quick Model, Feature Summary, Feature Details, Samples, and Definitions. You can export this report to share or review at a different time. Let’s look at some of the analysis in more detail. 
4. For **Target column**, select ***anomaly***.
> To determine good 5G accessibility, we will use abnormal_release_rate, which represents the likelihood of connectivity drops. Any abnormal_release_rate > 0 is considered high probability for anomaly. Then we will train a classification model that can predict the likelihood of connectivity drops based on input features like network utilization, contention rates, health index, and throughput parameters. This use case is part of the 5G performance observabilitinitiative, aimed at predicting any potential loss of connectivity to the 5G radio network in the next hour, helping to ensure a seamless and uninterrupted user experience.

5. For **Problem Type** select **Classification**.
6. Click the **Create** button to generate the report.

**Summary** provides a brief summary of the data that includes general information such as missing values, invalid values, feature types, outlier counts, and more. 
    
![Data Insights Summary](statics/dw-insights-summary.png)

**High Priority Warnings** lists warnings in the dataset if there are any and the steps we can take from within Data Wrangler to address them. 
    
![Data Insights Summary](statics/dw-insights-hp-warn.png)

**Duplicate Rows** helps you identify duplicate rows.
    
![Data Insights Duplicate Rows](statics/dw-insights-duplicate-rows.png)

**Anomalous Samples** are the most anomalous samples (with negative anomaly scores) identified by the Isolation forest algorithm. This helps you quickly spot outliers and anonymous data.
    
![Data Insights](statics/dw-insights-anom-samp.png)

**Target Column** analysis shows stats on target column and ranks the features on the order of their predictive power. It also detects potential issues and provides recommendation for remediation. **In this case, it has noticed your dataset is highly imbalance because anomalies are rare events. Recommendations are upsampling and class consolidation, which we will consider in our data preparation.**
    
![Data Insights Target Column](statics/dw-insights-target-col.png)

**Quick Model** provides an estimate of the expected predicted quality of a model that you train on your data. The Quick Model is a great way to get some prediction quality insight metrics on your dataset without going through the complete model building process.
    
![Data Insights Target Column](statics/dw-insights-quick-model.png)
    
![Data Insights Target Column](statics/dw-insights-quick-model-1.png)
    
![Data Insights Target Column](statics/dw-insights-quick-model-2.png)

## Feature Summary

Based on the target column, Data Wrangler orders the features in the Feature Summary by their prediction power. Scores are normalized to the range [0, 1]. Higher prediction scores indicate columns that are more useful for predicting the target on their own. Lower scores point to columns that aren’t predictive of the target column.
  
Notice some features are not contributing the to final prediction, and we can decide to drop these column base on our domain knowledge of this use case.
    
![Feature Summary](statics/feat-summary-1.png)
    
![Feature Summary](statics/feat-summary-2.png)

## Other Feature specific details and definitions

### Histogram

We want to understand the distribution of number of 5G users per hour:
    
![Histogram](statics/feat-summary-histogram.png)

1. Use the **< Data flow** button on the top left to get to the main data flow workspace.
2. Click the **“+”** icon next to the **Impute Anomaly** step in your data flow and select **Add analysis**.
3. Select **Histogram** as the **Analysis type**
4. Set the analysis name to *hist_number_of_5g_users*
5. Select the *number_of_5g_users* variable as the **X-axis**
6. Click the **Preview** button to visualize the results.
7. Click **Save** to save this analysis.

### Scatter plot

Let’s create a scatter plot to visualize the relationship between the number of users at cell towers and time (per hour). 
    
![Scatter Plot](statics/feat-summary-scatter-plot.png)

1. Choose **Scatter plot** as the **Analysis type**
2. For **Analysis Name** enter *scat_num_5g_users*
3. For X axis column, choose *hour_extracted*
4. For Y axis column, choose *number_of_5g_users*
5. Clicking on Preview yields the following visuals. Save the visualization by clicking on the Save button.

### Feature Correlation

Linear feature correlation is based on Pearson's correlation. Numeric to categorical correlation is calculated by encoding the categorical features as the floating point numbers that best predict the numeric feature before calculating Pearson's correlation. Linear categorical to categorical correlation is not supported.
  
To create the analysis, choose **Feature Correlation** for **Analysis** type and choose **linear** for **Correlation type**.
  
Based on the correlation values, we can see which feature pairs (as listed below) are strongly correlating with one another. Also, some of these features also showed up in the target analysis we did previously.
   
![Feature Correlation](statics/feat-correlation.png)

Please try the non-linear feature correlation on your own. Numeric to categorical correlation is calculated by encoding the categorical features as the floating point numbers that best predict the numeric feature before calculating Spearman's rank correlation. Categorical to categorical correlation is based on the normalized Cramer's V test.

### Custom Visualization

You can add an analysis to your Data Wrangler flow to create a custom visualization. Your dataset, with all applied transformations, is available as a [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

Data Wrangler uses the df variable to store the dataframe, which is accessible by the user. You must also provide the output variable, chart, to store an [Altair](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) output chart.

Take advantage of the example code snippets if you are not familiar with the altair library.  In this case, we are going to create a **scatter plot of Extracted Hour vs Throughput.**

1. For **Analysis type**, select *Custom Visualization*
2. For **Analysis name**, enter *throughput_vs_ hour*
3. Expand **Search example snippets** and select **Scatter plot**
4. Copy the code snippet to **Your custom visualization box**
5. Specify the column names for X and Y axis.

x="hour_extracted"\
y="5g_user_downlink_avg_throughput_num"

![Feature Summary](statics/feat-summary-custom-vis.png)

While Wait for the Data To Load, Let's Explore the Feature Store Console
Feature Store

![XXX](statics/feature_store.png)

## Drop Column

---

Let's drop some columns with low prediction power. To drop columns, we can choose the Drop column transform and pick the column names we want to drop as show in the image below.

1. Back to the **Data Flow**,  click + icon next to the **Impute Anomaly** step in your data flow and select **Add transform**.
    
![Drop Column](statics/feature-engg-drop-col-1.png)

2. Click **+ Add Step**.
    
![Drop Column](statics/feature-engg-drop-col-2.png)

3. Choose **Manage columns**.
4. **Transform** select **Drop column**.
5. **Columns to drop** select ***5g_nr_qos_flow_success_rate_num*** (you can choose multiple columns to drop).
6. Clicking on **Preview** to preview your data. Add the step to the workflow by clicking on the **Add** button.

In this step, we keep the following columns based on the above data insights and domain knowledge:
- *5g_avg_uplink_rssi*, 
- *5g_rrc_setup_success_rate_num*, 
- *5g_cce_utilization_num*, 
- *5g_rach_contention_rate_num*, 
- *number_of_5g_users*, 
- *cellname_nrcell*, 
- *hour_extracted*, 
- *anomaly*.

    
![Drop Column](statics/feature-engg-drop-col-3.png)

## Handle Categorical Features
---

Categorical data is usually composed of a finite number of categories, where each category is represented with a string. Ordinal categories have an inherent order, and nominal categories do not. The Machine Size Type (L, M, H) is an example of ordinal categories.

Encoding categorical data is the process of creating a numerical representation for categories. There are 3 ways we can encode a categorical value in Data Wrangler.

* Ordinal encode
* One Hot encode
* Similarity encode

With Data Wrangler, we can select Ordinal encode to encode categories into an integer between 0 and the total number of categories in the Input column you select. Select One-hot encode for Transform to use one-hot encoding or use similarity encoding when you have the following:

* A large number of categorical variables
* Noisy data

Here, let’s apply ordinal encode for *cell_ID*.

1. Click **Add step**.
2. Select **Encode categorical**.
3. **Transform** select Ordinal encode.
4. From the **Input columns** select *cellname_nrcell*.
5. Click **Preview** and then **Add**.

Note: without defining an **Output column**, this is going to perform an inplace transform. If you need to keep the original column, please provide an **Output column**.
    
![Handle Categorical Features](statics/handle-cat-features.png)

## Fill & Drop Missing
---

### Fill Missing

We now fill missing values for numeric features:

1. Add step.
2. Select **Handle missing**.
3. **Transform** select **Fill missing**.
4. **Input columns** select numeric columns
5. **Fill value** is 0
6. Click **Preview** and then **Add**

![Normalize Numeric Features](statics/fill-drop-missing-1.png)

### Drop Missing

Let us drop rows with missing values:

1. Add step.
2. Select **Handle missing**.
3. **Transform** select **Drop missing**.
4. Do not provide **Input columns**, so that all rows with a missing value in any column will be removed.
5. Click **Preview** and then **Add**.

![Normalize Numeric Features](statics/fill-drop-missing-2.png)

## Normalize Numeric Features

---

Machine learning algorithms like linear regression, logistic regression, neural networks, etc. that use gradient descent as an optimization technique require data to be scaled. To ensure that the gradient descent moves smoothly towards the minima and that the steps for gradient descent are updated at the same rate for all the features, we scale the data before feeding it to the model. Having features on a similar scale can help the gradient descent converge more quickly towards the minima.

Normalization is good to use when you know that the distribution of your data does not follow a Gaussian distribution. This can be useful in algorithms that do not assume any distribution of the data like K-Nearest Neighbors and Neural Networks. Standardization, on the other hand, can be helpful in cases where the data follows a Gaussian distribution. However, this does not have to be necessarily true. Also, unlike normalization, standardization does not have a bounding range. So, even if you have outliers in your data, they will not be affected by standardization.

For this example use case, let's see how to normalize the numeric feature columns to a standard scale [0, 1].

1. Click **Add step**.
2. Select **Process numeric**.
3. **Transform** select *Scale values*.
4. **Scaler** select *Min-max scaler*.
5. **Input columns** select ***numeric columns***.
6. Select **Scale**.
7. Click **Preview** and then **Add**.

![Normalize Numeric Features](statics/normalize-num-features.png)

## Custom Transform
---

Rename columns to make them easier to workwith.

1. Click Add step
2. Select Custom transform
3. Name put Rename Columns
4. Select Python (Pandas)
5. Place following code snippet into the code box

```
# Table is available as variable `df`
column_names = {
  "cellname_nrcell":"location_id",
  "number_of_5g_users":"5g_users",
  "5g_rach_contention_rate_num":"contention_rate",
  "5g_rrc_setup_success_rate_num":"rrc_success_rate",
  "5g_avg_uplink_rssi":"uplink_rssi",
  "5g_cce_utilization_num":"cce_utilization",
  "5g_cell_downlink_avg_throughput_num":"downlink_throughput",
 "5g_cell_uplink_avg_throughput_num":"uplink_throughput",
}
df = df.rename(columns=column_names)
```

![Rename Columns](statics/fe-rename-cols.png)

## Data Destination
---

You will be exporting the clean features into S3 and Amazon SageMaker Feature Store for the following lab 2 and 3 respectively.

At this point, your data wrangler workflow should look something like this:

![Add Destination](statics/add-destination-0.png)

Let’s create a data destination for Feature Store first:

1. Click the **“+”** from the node you wish to export from. All transforms made before, up to, and including that node will be included in the export.
2. Choose **Add destination**.
3. Choose Add destination. You can choose S3 or Feature Store, but for our example, we'll select **Feature Store**.

![Add Destination](statics/add-destination-1.png)

4. Select **5gcell-anomaly-features** from the feature group list. 
5. Click on **“Click this message to .....”** to validate the data schema. Follow the instruction to add **event time column**.
6. Click Add to add feature group as data destination.

![Add Destination](statics/add-destination-2.png)

**Note: At this point, the columns in your data flow has to match the schema in the feature store. If you are getting a miss match, please carefully trace back the steps and recheck your work.**


Follow a similar process to create a data destination with S3:

1. Click the **'+'** from the node you wish to export from. All transforms made before, up to, and including that node will be included in the export.
2. Choose **Add destination**. You can choose S3 or Feature Store, but for our example, we'll select S3.
3. The *Add Destination* panel appears on the right side of Studio.
    1. dataset name *(5gcell-clean.csv)* 
    2. S3 location: *s3://sagemaker-<Region>-<Account_ID>/telco-5g-observability/data/clean*

## Process Full Data
---

Now we are ready to execute the entire data flow to process our data end-to-end:

1. Click **Create job**.  

![Process Data](statics/process-data-1.png)

The *Create job* panel appears from the right side with a default job name already populated. There are also *KMS*, and *Refit* trained parameter options.
If you were encrypting the output data with KMS, you could enter that key arn here. For the purposes of this workshop, we'll keep it blank.

2\. Leave all other fields default, and click Next, **Configure Job**.\
The *Configure* job panel is now displayed.
When we execute the job, the processing run parallelized across multiple EC2 instances. Based on the size of your dataset and the complexity of the transforms, you may wish to select a different instance type and count to improve speed and performance. For now, we'll keep the defaults.  
Under the *Spark memory configuration* tab, we can override default driver and executor memory settings. For now, we'll keep the defaults.  
Under the *Parameters* tab, we see there is the *basename_param* we had defined earlier. If you click on it, you can change the value. Since this is the correct path we wish to use, we'll leave the default.
If you're processing data periodically, you can create a schedule to run the processing job automatically. For example, you can create a schedule that runs a processing job automatically when you get new data. This is configured under the *Associate Schedules* tab.  
For more information on creating schedules, please see [Create a Schedule to Automatically Process New Data](https://docs.aws.amazon.com/sagemaker/latest/dg/data-wrangler-data-export.html#data-wrangler-data-export-schedule-job).

3. Click **Create** to create the processing job. The confirmation screen will appear.

![Process Data](statics/process-data-2.png)

You can click on the *Processing Job* name in the confirmation dialog to monitor the state of the export. Once completed, your data will be saved to both Feature store and S3 destinations you defined.

## [Optional] Train & Test Split
---

** Note: This section of the lab is optional.**

You can also train a model from SageMaker Data Wrangler. This feature uses Amazon SageMaker Autopilot to automatically train, tune, and deploy models on the data that you've just transformed. Underneath the hood, Autopilot goes through several algorithms and use the one that works best with your data.
Now we want to split our train and test set. We can do this using the Split data transform.

To train a model using Autopilot,  you will first split the data into train and test set.

1. Click **Add step**.
2. Select **Split data**.
3. **Transform** select *Randomized split*.
4. Train: *0.8*, Test: *0.2*.
5. Click **Preview** and then **Add**.

![Train & Test Split](statics/train-test-1.png)

For the purpose of this lab, we are going to use the Test dataset for batch inference, so we will add one more step to remove the target column. Click the **“+”** icon near Dataset: **5gcell.csv (Test)**

1. Click **Add transform** (Test data only).
2. Click **Add step**.
3. select **Manage columns**.
4. **Transform** select **Drop column**.
5. **Columns to drop** select *anomaly*.
6. Click **Preview** and then **Add**.

![Train & Test Split](statics/train-test-2.png)

Your data flow should now be split into 2 datasets branches. For each of the branch, let’s add an S3 destination following the previous instructions.

- For train dataset: 

    - dataset name (*train.csv*), and the 
    - S3 location: *s3://sagemaker-<Region>-<Account_ID>/telco-5g-observability/data/train*.
    - Leave everything else default, then click **Add destination**. 
    
* For test dataset: 
    
    * dataset name (*test.csv*)
    * S3 location: *s3://sagemaker-<Region>-<Account_ID>/telco-5g-observability/data/test*.
    * Leave everything else default, then click **Add destination**. 

Your flow should looks like this. If you are at this point, Create a new Job to transform your data.

![Train & Test Split](statics/train-test-3.png)


## [Optional] Train a Model with Autopilot
---

**Note: This section of the lab is optional.**

When you train and tune a model, Data Wrangler exports your data to an Amazon S3 location where Amazon SageMaker Autopilot can access it.

1. Choose the **“+”** next to the train dataset, and select **Train model**.
2. For Amazon S3 location, specify the Amazon S3 location where SageMaker exports your data. If presented with a root bucket path by default, Data Wrangler will create a unique export sub-directory under it — you don’t need to modify this default root path unless you’d like to.

![Train Autopilot](statics/train-autopilot-1.png)

You can accept the defaults, and click the *Export* and *train* button to export the transformed data to S3.

![Train Autopilot](statics/train-autopilot-2.png)


Once export is successful, you are taken to the Create an **Autopilot experiment** page, with the Input data S3 location already filled in for you (as it was populated from the results of the previous screen.)

3. Optionally, set an Experiment name (if you don’t want the default name.)

![Train Autopilot](statics/train-autopilot-3.png)


4. Click **Next: Target and features**.
5. **Target** select anomaly.
6. Click **Next: Training method**.

![Train Autopilot](statics/train-autopilot-4.png)

As detailed in the post Amazon SageMaker Autopilot is up to eight times faster with new ensemble training mode powered by AutoGluon, you can either let Autopilot select the training mode automatically based on the dataset size, or select the training mode manually for either ensembling or hyperparameter optimization (HPO).
The details of each option are as follows:

* **Auto** – Autopilot automatically chooses either ensembling or HPO mode based on your dataset size. If your dataset is larger than 100 MB, Autopilot chooses HPO; otherwise it chooses ensembling.
* **Ensembling** – Autopilot uses the AutoGluon ensembling technique to train several base models and combines their predictions using model stacking into an optimal predictive model.
* **Hyperparameter optimization** – Autopilot finds the best version of a model by tuning hyperparameters using the Bayesian optimization technique and running training jobs on your dataset. HPO selects the algorithms most relevant to your dataset and picks the best range of hyperparameters to tune the models.

7. For our workshop example, we'll leave the default selection of Auto.
8. Click **Next: Deployment and advanced** settings to continue.
9. For **Deployment settings**, switch to **Yes** for **Auto deploy**. Input **Auto deploy endpoint name**. 
10. **Select the machine learning problem type** to be **Binary classification**. **Objective metric** select F1.
11. Click **Next: Review and create**.

![Train Autopilot](statics/train-autopilot-5.png)

12. Click **Create experiment**.

![Train Autopilot](statics/train-autopilot-6.png)

13. The results are listed and sorted in the decreasing order of the objective (F1) score. It also highlights the best performing model.

![Train Autopilot](statics/train-autopilot-7.png)

Click **View model details, Explainbility** tab to show **FEATURE IMPORTANCE**. You  can select explanations for **“0.0”** or **“1.0”**. 

![Train Autopilot](statics/train-autopilot-8.png)

Finally, with just a few clicks, you can **Deploy** any of your Autopilot experiments, either online to an realtime endpoint or offline for batch predictions.

![Train Autopilot](statics/train-autopilot-9.png)
